<a href="https://colab.research.google.com/github/envirodatascience/final-project-insect-team/blob/main/ENV617_group_project_ypm_ent_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Doing some summarizing, cleaning of YPM entomology data

In [3]:
# need to install pygbif module
!pip install pygbif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.4 MB/s eta 0:00:00


In [4]:
# import modules
import pandas as pd
import numpy as np

from pygbif import occurrences as occ # I think we only need the occurrences module

from google.colab import userdata # for accessing username and password stored in environment variables

In [18]:
# download ypm ent data
ent = occ.download(['institutionCode = YPM', 'collectionCode = ENT'], pred_type ='and',
                      user=userdata.get('GBIF_USER'), pwd=userdata.get('GBIF_PWD'), email='portmajm@gmail.com') # i think these should be able to be read in without stating?

In [20]:
ent # will send an email when the download is ready

('0000426-250402121839773',
 {'creator': 'portmajm',
  'notification_address': ['portmajm@gmail.com'],
  'sendNotification': True,
  'predicate': {'type': 'and',
   'predicates': [{'type': 'equals',
     'key': 'INSTITUTION_CODE',
     'value': 'YPM'},
    {'type': 'equals', 'key': 'COLLECTION_CODE', 'value': 'ENT'}]},
  'format': 'SIMPLE_CSV'})

In [ ]:
occ.get_download(key='0000426-250402121839773')

In [11]:
!ls

0000426-250402121839773.zip  sample_data


In [12]:
!unzip 0000426-250402121839773.zip

Archive:  0000426-250402121839773.zip
  inflating: 0000426-250402121839773.csv  


In [13]:
!ls

0000426-250402121839773.csv  0000426-250402121839773.zip  sample_data


In [16]:
df_ent = pd.read_csv('0000426-250402121839773.csv', index_col = False,sep="\t")

<ipython-input-16-ea43a1fd9988>:1: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ent = pd.read_csv('0000426-250402121839773.csv', index_col = False,sep="\t")


In [17]:
df_ent.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,3913895301,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:00006775-753e-4ed0-bbe6-0702cae6c26b,Animalia,Arthropoda,Insecta,Lepidoptera,Papilionidae,Papilio,Papilio polyxenes,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,R. T. Cail,NaN,NaN,2025-04-01T13:31:34.353Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
1,1438048515,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:000426c2-4fff-44bf-97c3-461d8022c177,Animalia,Arthropoda,Insecta,Lepidoptera,Noctuidae,Achatia,Achatia evicta,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,Richard E. Gray,NaN,NaN,2025-04-01T13:31:34.410Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
2,1838432720,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:00091984-cf96-4498-a6c6-e034f72a355a,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Lobophora,Lobophora nivigerata,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,David S. MacDonald,NaN,NaN,2025-04-01T13:31:25.822Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
3,1935872503,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:0009a4ea-0d53-46da-bf70-949145977b1b,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Metarranthis,Metarranthis amyrisaria,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,"Timothy L. McCabe, Dale F. Schweitzer",NaN,NaN,2025-04-01T13:31:34.328Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
4,2565549387,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:000a73bd-10b3-46d3-b38c-be8c3d909b83,Animalia,Arthropoda,Insecta,Lepidoptera,Crambidae,Parapoynx,Parapoynx obscuralis,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,Herman P. Wilhelm,NaN,NaN,2025-04-01T13:31:34.426Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...


In [18]:
df_ent.shape()

TypeError: 'tuple' object is not callable

In [19]:
df_ent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421950 entries, 0 to 421949
Data columns (total 50 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   gbifID                            421950 non-null  int64  
 1   datasetKey                        421950 non-null  object 
 2   occurrenceID                      421950 non-null  object 
 3   kingdom                           421950 non-null  object 
 4   phylum                            421438 non-null  object 
 5   class                             421424 non-null  object 
 6   order                             421394 non-null  object 
 7   family                            420948 non-null  object 
 8   genus                             415811 non-null  object 
 9   species                           392092 non-null  object 
 10  infraspecificEpithet              3666 non-null    object 
 11  taxonRank                         421950 non-null  o

In [22]:
df_ent.value_counts('order')

,count
order,
Lepidoptera,349114
Hymenoptera,22832
Diptera,19248
Coleoptera,15547
Odonata,6428
Hemiptera,3655
Ixodida,1056
Trichoptera,773
Neuroptera,601


In [21]:
df_ent[df_ent.stateProvince == 'Connecticut'].value_counts('order')

,count
order,
Lepidoptera,51071
Hymenoptera,4437
Diptera,3486
Coleoptera,2891
Odonata,1020
Trichoptera,712
Neuroptera,508
Hemiptera,478
Megaloptera,221


In [26]:
df_ent[df_ent.family == 'Nymphalidae']

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
5,1253318886,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:000ac4e3-665a-41d5-a146-5a86bde1041c,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Asterocampa,Asterocampa celtis,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,"Dale F. Schweitzer, Howard D. Baggett",NaN,NaN,2025-04-01T13:31:34.429Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
8,1039025623,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:000fa4f1-39fb-46f5-9ed1-ac056fca8d58,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Limenitis,Limenitis boreas,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,NaN,NaN,NaN,2025-04-01T13:31:25.880Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
20,4528475301,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:001ae868-44db-4f59-ae23-96fc4077c79e,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Oeneis,Oeneis bore,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,John W. Peacock,NaN,NaN,2025-04-01T13:31:25.895Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
22,1039025668,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:001b5796-233d-4d04-adcf-f12e6bc6b5c5,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Cercyonis,Cercyonis pegala,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,Edward Force,NaN,NaN,2025-04-01T13:31:34.490Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
32,1039025757,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:00309dc3-eff7-43b2-b0af-67688404beb8,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Danaus,Danaus chrysippus,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,A. J. Good,NaN,NaN,2025-04-01T13:31:34.425Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421927,4536154364,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:ffe358f7-7558-49e4-b5b1-ec84d53f3e9c,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Coenonympha,Coenonympha tullia,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,"John W. Peacock, Ruth A. Peacock",NaN,NaN,2025-04-01T13:32:02.384Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
421928,1039205252,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:ffe3bd54-8771-457d-80cd-cf7fdf55409c,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Acraea,Acraea encedon,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,Stanislaw Chojnacki,NaN,NaN,2025-04-01T13:32:02.384Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
421941,1039205414,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:fff5f1e2-0aa9-48ac-a65f-284d20b5908a,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Eresia,NaN,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,"Kirby W. Brown, D. Veirs",NaN,NaN,2025-04-01T13:32:16.610Z,StillImage,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...
421945,1039205035,96404cc2-f762-11e1-a439-00145eb45e9a,urn:uuid:fffd79f3-938c-43a6-97fd-a178a69dfbe5,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Acraea,Acraea atergatis,...,NaN,NaN,CC0_1_0,Yale Peabody Museum,Gerd H. Heinrich,NaN,NaN,2025-04-01T13:32:16.612Z,NaN,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...


In [28]:
pd.to_csv(df_ent, 'C:/Users/live2/OneDrive - Yale University/Documents/_Yale/classes/SP25/ENV_617/ypm_insect_data_all.csv')

AttributeError: module 'pandas' has no attribute 'to_csv'